In [5]:
import numpy as np
import logging
import argparse
import multiprocessing as mp
from time import time
from tensorflow.keras.layers import Dense, Flatten, Input, Embedding, Multiply, Concatenate
from tensorflow.keras import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import tensorflow as tf

In [1]:
import numpy as np
import pandas as pd
import os

In [6]:

def get_neumf_model(num_users, num_items, layers=[128, 64, 32, 16], reg=["l1"]):
    user_input = Input(shape=(1,), dtype="int32", name="user_input")
    item_input = Input(shape=(1,), dtype="int32", name="item_input")

    gmf_emb_user = Embedding(input_dim=num_users, output_dim=emb_dim,
                             name="gmf_user_emb", input_length=1)
    gmf_emb_item = Embedding(input_dim=num_items, output_dim=emb_dim,
                             name="gmf_item_emb", input_length=1)
    gmf_user_emb = Flatten()(gmf_emb_user(user_input))
    gmf_item_emb = Flatten()(gmf_emb_item(item_input))
    gmb_vector_layer = Multiply()([gmf_user_emb, gmf_item_emb])

    mlp_emb_user = Embedding(input_dim=num_users, output_dim=int(layers[0] / 2),
                             name="mlp_user_emb", input_length=1)
    mlp_emb_item = Embedding(input_dim=num_items, output_dim=int(layers[0] / 2),
                             name="mlp_item_emb", input_length=1)
    mlp_user_emb = Flatten()(mlp_emb_user(user_input))
    mlp_item_emb = Flatten()(mlp_emb_item(item_input))
    mlp_vector_layer = Concatenate()([mlp_user_emb, mlp_item_emb])

    for idx in range(1, len(layers)):
        mlp_vector_layer = Dense(layers[idx], activation='relu', kernel_initializer="lecun_uniform",
                                 name="mlp_layer_" + str(idx))(mlp_vector_layer)

    predict_vector = Concatenate()([gmb_vector_layer, mlp_vector_layer])
    prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name='prediction')(predict_vector)
    model = Model(inputs=[user_input, item_input],
                  outputs=prediction)
    return gmf_emb_user, gmf_emb_item, model


In [7]:
num_users = 31568
num_items = 9065
emb_dim = 64
user_emb, item_emb, model = get_neumf_model(num_users, num_items)

In [9]:
model.load_weights("/Users/wizardholy/project/recsys_learning/Pretrain/info_gmf_64_[64,32,16,8]_1585834028.h5")

In [11]:
print(model.get_layer("mlp_user_emb"))

In [29]:
embs = model.get_layer("mlp_user_emb").embeddings.numpy()

In [37]:
for i in range(10):
    print(i,"\t",",".join([str(item) for item in embs[i].tolist()]))

0 	 0.09206975996494293,0.09069786965847015,0.08962332457304001,0.09389825165271759,0.02131730690598488,0.15799246728420258,-0.03746502846479416,-0.004116828087717295,-0.1681564748287201,-0.09434854239225388,0.059654463082551956,-0.0533372163772583,-0.10654091089963913,0.02340487577021122,-0.055833850055933,0.021917501464486122,0.038392502814531326,-0.03437141329050064,0.0070918528363108635,-0.0013900332851335406,0.0550147220492363,-0.08550190180540085,-0.1493201106786728,-0.13458579778671265,0.1583864390850067,0.13046889007091522,-0.18480290472507477,0.021051794290542603,-0.12448117136955261,-0.11394751071929932,0.03052767738699913,0.007067359518259764,0.10216405242681503,-0.13252593576908112,0.06849794834852219,0.1338813304901123,0.007615786045789719,-0.08994075655937195,0.07811178267002106,-0.08759474754333496,-0.09257994592189789,-0.03680018335580826,-0.1149737685918808,0.14643600583076477,0.02035592310130596,-0.01253474224358797,-0.10968095064163208,-0.14384044706821442,0.08394173